# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A,B를 랜덤 추출한다
4. A,B와 대응되는 C를 추출한다.
5. D를 입력받는다
=> A:B = C:answer 관계에 대응하는 answer를 찾는 게임을 만든다
ex) A:산, B:바다, C:나무/ D-> 물

In [3]:
!pip install wikipedia-api


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [5]:
import wikipediaapi
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
import konlpy
from konlpy.tag import Okt
import tqdm
import re

In [ ]:
wiki = WikiCorpus('kowiki-latest-pages-articles.xml.bz2', dictionary={})

okt = Okt()

def preprocess_text(text):
    return okt.morphs(text)


max_docs = 3000

sentences = []
for i, text in tqdm.tqdm(enumerate(wiki.get_texts()), desc="Processing documents"):
    if i >= max_docs:  # 지정된 수만큼만 처리
        break
    text = ' '.join(text)  # text를 문자열로 변환
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)  # 한글과 공백만 남기기
    tokens = preprocess_text(text)
    if tokens:  # 빈 리스트가 아닐 경우만 추가
        sentences.append(tokens)
    
    if i % 1000 == 0:
        print(f"Processed {i} documents")

print(f"Total processed documents: {len(sentences)}")

Processing documents: 0it [00:00, ?it/s]/usr/local/lib/python3.10/site-packages/gensim/utils.py:1332: UserWarning: detected OSX with python3.8+; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)
Processing documents: 1it [00:03,  3.29s/it]

Processed 0 documents


Processing documents: 1000it [02:37,  4.32it/s]

Processed 1000 documents


Processing documents: 2006it [04:34, 20.67it/s]

Processed 2000 documents


Processing documents: 3000it [06:47,  7.36it/s]

Total processed documents: 3000


In [8]:
model = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=0
)
model.save('wiki_word2vec.model')

In [ ]:
import random
import numpy as np

# 모델 로드
model = Word2Vec.load('wiki_word2vec.model')

def find_similar_pairs(model, similarity_threshold=0.8, max_attempts=1000):
    """유사도가 threshold 이상인 단어 쌍을 찾습니다"""
    vocabulary = model.wv.index_to_key
    
    while max_attempts > 0:
        word_a = random.choice(vocabulary)
        word_b = random.choice(vocabulary)
        
        try:
            similarity = model.wv.similarity(word_a, word_b)
            if similarity >= similarity_threshold:
                return word_a, word_b
        except KeyError:
            continue
        
        max_attempts -= 1
    
    return None, None

def word_analogy_game():
    """단어 유추 게임을 실행합니다"""
    # 유사한 단어 쌍 찾기
    word_a, word_b = find_similar_pairs(model)
    if word_a is None:
        print("적절한 단어 쌍을 찾을 수 없습니다.")
        return
    
    # C 단어 선택 (A와 유사하지 않은 단어)
    while True:
        word_c = random.choice(model.wv.index_to_key)
        try:
            if model.wv.similarity(word_a, word_c) < 0.3:  # 낮은 유사도
                break
        except KeyError:
            continue
    
    print(f"\n단어 유추 게임!")
    print(f"{word_a} : {word_b} = {word_c} : ?")
    
    answer = input("답을 입력하세요: ")
    
    try:
        # 정답 평가
        result = model.wv.most_similar(
            positive=[word_b, word_c],
            negative=[word_a],
            topn=3
        )
        
        print("\n가장 적절한 답들:")
        for word, score in result:
            print(f"{word}: {score:.3f}")
        
        # 사용자의 답 평가
        if answer in [word for word, _ in result]:
            print("\n정답입니다!")
        else:
            try:
                user_answer_score = model.wv.similarity(answer, result[0][0])
                if user_answer_score > 0.5:
                    print(f"\n괜찮은 답변입니다! (유사도: {user_answer_score:.3f})")
                else:
                    print("\n다른 답을 생각해보세요.")
            except KeyError:
                print("\n모델이 이 단어를 모릅니다.")
                
    except KeyError:
        print("죄송합니다. 이 단어들로는 유추가 어렵습니다.")

# 게임 실행
while True:
    word_analogy_game()
    
    play_again = input("\n다시 하시겠습니까? (y/n): ")
    if play_again.lower() != 'y':
        break


단어 유추 게임!
파전 : 홍문종 = 저지 : ?

가장 적절한 답들:
사수: 0.802
협조: 0.796
해임: 0.796

모델이 이 단어를 모릅니다.


In [5]:
import random
import numpy as np
import time
import os
from gensim.models import Word2Vec

# 모델 로드
model = Word2Vec.load('wiki_word2vec.model')

def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

def print_with_delay(text, delay=0.03):
    for char in text:
        print(char, end='', flush=True)
        time.sleep(delay)
    print()

def dramatic_dots(count=3, delay=0.5):
    for _ in range(count):
        print(".", end='', flush=True)
        time.sleep(delay)
    print()

def find_similar_pairs(model, similarity_threshold=0.7, max_attempts=1000):
    """유사도가 threshold 이상인 단어 쌍을 찾습니다"""
    vocabulary = model.wv.index_to_key
    
    while max_attempts > 0:
        word_a = random.choice(vocabulary)
        word_b = random.choice(vocabulary)
        
        try:
            similarity = model.wv.similarity(word_a, word_b)
            if similarity >= similarity_threshold:
                return word_a, word_b
        except KeyError:
            continue
        
        max_attempts -= 1
    
    return None, None

def pokemon_style_word_analogy_game():
    clear_screen()
    print_with_delay("\n야생의 단어 쌍이 나타났다!")
    time.sleep(1)

    # 유사한 단어 쌍 찾기
    word_a, word_b = find_similar_pairs(model, similarity_threshold=0.7)
    if word_a is None:
        print_with_delay("앗! 단어 쌍이 도망가버렸다...")
        return

    print_with_delay(f"\n야생의 (A)'{word_a}'와(과) (B)'{word_b}'가 나타났다!", 0.05)
    time.sleep(1)
    
    print_with_delay("\n두근두근...", 0.1)
    dramatic_dots()
    
    # C 단어 선택
    while True:
        word_c = random.choice(model.wv.index_to_key)
        try:
            if model.wv.similarity(word_a, word_c) < 0.3:
                break
        except KeyError:
            continue

    print_with_delay(f"\n갑자기 (C)'{word_c}'가 나타났다!", 0.05)
    time.sleep(1)
    
    print_with_delay("\n어떤 단어와 관련이 있을까...?", 0.05)
    dramatic_dots(3, 0.3)
    
    answer = input("\n당신의 추측은? : ")
    
    print_with_delay("\n분석중", 0.1)
    dramatic_dots(3, 0.3)
    
    try:
        result = model.wv.most_similar(
            positive=[word_b, word_c],
            negative=[word_a],
            topn=3
        )
        
        print_with_delay("\n가장 적절한 답들이야!", 0.05)
        time.sleep(0.5)
        for word, score in result:
            print_with_delay(f"{word}: {score:.3f}", 0.03)
        
        if answer in [word for word, _ in result]:
            print_with_delay("\n와! 정확한 답이야!", 0.05)
            print_with_delay("게임의 승자가 되었다!", 0.05)
        else:
            try:
                user_answer_score = model.wv.similarity(answer, result[0][0])
                if user_answer_score > 0.5:
                    print_with_delay(f"\n꽤 괜찮은 시도였어! (유사도: {user_answer_score:.3f})", 0.05)
                else:
                    print_with_delay("\n아쉽게도 틀렸어... 다시 도전해봐!", 0.05)
            except KeyError:
                print_with_delay("\n그런 단어는 모르겠어...", 0.05)
                
    except KeyError:
        print_with_delay("앗! 뭔가 잘못된 것 같아...", 0.05)

# 게임 실행
while True:
    pokemon_style_word_analogy_game()
    time.sleep(1)
    play_again = input("\n다시 도전하시겠습니까? (y/n): ")
    if play_again.lower() != 'y':
        print_with_delay("\n게임을 종료합니다... 다음에 또 만나요!", 0.05)
        break


야생의 단어 쌍이 나타났다!

야생의 (A)'미코'와(과) (B)'엘바'가 나타났다!

두근두근...
...

갑자기 (C)'않았을까'가 나타났다!

어떤 단어와 관련이 있을까...?
...

분석중
...

가장 적절한 답들이야!
허락: 0.730
않겠다고: 0.720
않도록: 0.720

그런 단어는 모르겠어...

야생의 단어 쌍이 나타났다!

야생의 (A)'미노국'와(과) (B)'간문제'가 나타났다!

두근두근...
...

갑자기 (C)'받았으며'가 나타났다!

어떤 단어와 관련이 있을까...?
...

분석중
...

가장 적절한 답들이야!
받았고: 0.774
받았다: 0.669
받았으나: 0.643

아쉽게도 틀렸어... 다시 도전해봐!

게임을 종료합니다... 다음에 또 만나요!
